In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import re
from tqdm import tqdm
import csv

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [3]:
# create a service object for interacting with the YouTube Data API
youtube = build("youtube", "v3", developerKey=api_key, cache_discovery=False)

In [4]:
def extract_video_id(url):
    video_id = None
    if "youtu.be" in url:
        video_id = url.split("/")[-1]
    else:
        query_string = url.split("?")[-1]
        parameters = query_string.split("&")
        for param in parameters:
            if param.startswith("v="):
                video_id = param[2:]
                break

    return video_id

In [8]:
# TEST
link = "https://www.youtube.com/watch?v=36N1Bz7qW0A"
request = youtube.commentThreads().list(part='id,replies,snippet', videoId=extract_video_id(link))
response = request.execute()
print(response)

In [9]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
from tqdm import tqdm

# Load your API key from an environment variable
load_dotenv()
api_key = os.getenv("API_KEY")

# Create a service object for interacting with the YouTube Data API
youtube = build("youtube", "v3", developerKey=api_key, cache_discovery=False)

def extract_video_id(url):
    # Assuming the URL is in the format 'https://www.youtube.com/watch?v=video_id'
    return url.split('v=')[1]

def process_comments(response_items, comments_df):
    for res in response_items:
        if 'replies' in res:
            # Process replies to top-level comments
            for reply in res['replies']['comments']:
                comment = reply['snippet']
                comment['parentId'] = res['snippet']['topLevelComment']['id']
                comments_df = comments_df.append(comment, ignore_index=True)
        else:
            # Top-level comment
            comment = res['snippet']['topLevelComment']['snippet']
            comment['parentId'] = None
            comments_df = comments_df.append(comment, ignore_index=True)
    return comments_df

def get_all_comments(video_id):
    comments_df = pd.DataFrame()  # Initialize an empty DataFrame

    request = youtube.commentThreads().list(
        part='id,replies,snippet',
        videoId=video_id,
        maxResults=100
    )

    while request:
        response = request.execute()

        # Create DataFrame columns dynamically based on response keys
        if not comments_df.columns:
            comments_df = pd.DataFrame(columns=response['items'][0]['snippet'].keys())

        comments_df = process_comments(response['items'], comments_df)

        if 'nextPageToken' in response:
            request = youtube.commentThreads().list_next(request, response)
        else:
            break

    return comments_df

def main():
    video_ids = ['36N1Bz7qW0A']

    for video_id in tqdm(video_ids, desc='Processing videos'):
        all_comments_df = get_all_comments(video_id)

if __name__ == '__main__':
    main()

Processing videos:   0%|                                                                         | 0/1 [00:00<?, ?it/s]


ValueError: The truth value of a RangeIndex is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().